### Necessary modules

Before processing the analysis, user should install some libraries in Prompt.
```ruby: Anaconda Prompt
! pip install mecab-python3
! pip install unidic-lite
! pip install --no-binary :all: mecab-python3
! conda install  scikit-learn
```

In [33]:
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd

### データのサンプル例を作成、textは新聞記事、categoryは分類ラベル。

In [48]:
data = {
    'text': ['日本経済成長が鈍化', '東京オリンピックが開幕', '技術革新が産業発展を推進', '宇宙探査の新たな里程碑',
             '日本の経済文化について', '新型コロナウイルスの影響', '人工知能の未来', '自然エネルギーの重要性',
             '地球温暖化と健康', '最新の医療技術', '宇宙開発の進展', '人口問題と社会政策',
             '経済成長と環境保護', 'スポーツの未来展望', '新しい教育システム', '国際貿易と平和の重要性'],
    'category': ['経済', '経済', '科学技術', '科学技術',
                 '経済', '経済', '科学技術', '経済',
                 '経済', '経済', '科学技術', '経済',
                 '経済', '経済', '経済', '経済']
}
df = pd.DataFrame(data)
df.head()

,text,category
0,日本経済成長が鈍化,経済
1,東京オリンピックが開幕,経済
2,技術革新が産業発展を推進,科学技術
3,宇宙探査の新たな里程碑,科学技術
4,日本の経済文化について,経済


### Step 1: MeCabを使用してテキストのトークン化と形態素解析を行う関数を定義

In [49]:
def tokenize(text):
    tagger = MeCab.Tagger('-Owakati')  # 分かち書き出力を使用
    result = tagger.parse(text)
    return result.strip().split()  # 分かち書きされた単語のリストを返す

# Usage:
sample_text = "日本経済が回復"
token = tokenize(sample_text)
print(token)


['日本', '経済', 'が', '回復']


### Step 2: TF-IDFベクトル化器とナイーブベイズ分類器（Multinomial Naive Bayes）を含むパイプラインを作成

In [50]:
# テキストとラベルをそれぞれXとyとして格納
X = df['text']
y = df['category']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"教師新聞と分類: \n {X_train.head()} \n {y_train.head()}")
print(f"\n テスト用新聞と分類: \n {X_test.head()} \n {y_test.head()}")

# TF-IDFベクトル化器とナイーブベイズ分類器（Multinomial Naive Bayes）を含むパイプラインを作成
model = make_pipeline(TfidfVectorizer(tokenizer=tokenize, token_pattern=None), MultinomialNB())

教師新聞と分類: 
 13       スポーツの未来展望
11       人口問題と社会政策
8         地球温暖化と健康
9          最新の医療技術
2     技術革新が産業発展を推進
Name: text, dtype: object 
 13      経済
11      経済
8       経済
9       経済
2     科学技術
Name: category, dtype: object

 テスト用新聞と分類: 
 0        日本経済成長が鈍化
1      東京オリンピックが開幕
5     新型コロナウイルスの影響
14       新しい教育システム
Name: text, dtype: object 
 0     経済
1     経済
5     経済
14    経済
Name: category, dtype: object


### Step 3: モデルを訓練、予測、正解率を出す

サンプル例が少なすぎるため、正解率は低いが、これで記事に対する分類の流れがまとめられた。

In [51]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("正解率:", metrics.accuracy_score(y_test, y_pred))
print(f"正解: {y_test.values.tolist()}")
print(f"予測: {y_pred}")


正解率: 1.0
正解: ['経済', '経済', '経済', '経済']
予測: ['経済' '経済' '経済' '経済']
